In [1]:
from seamless.highlevel import Context, Transformer

ctx = Context()

In [2]:
ctx.pdb1 = open("1AKE-flanks.pdb").read()
ctx.pdb2 = open("1AKE-B-hit.pdb").read()
ctx.load_pdb1 = Transformer()
ctx.load_pdb1.pdb = ctx.pdb1
ctx.load_pdb_code >> ctx.load_pdb1.code
ctx.load_pdb_code.mount("load_pdb.py")
ctx.coor1 = ctx.load_pdb1
ctx.load_pdb2 = Transformer()
ctx.load_pdb2.pdb = ctx.pdb2
ctx.load_pdb2.code = ctx.load_pdb_code
ctx.coor2 = ctx.load_pdb2

In [3]:
def bcscore(coor1, coor2, flanksize, gapsize):
    print("BCSCORE", flanksize, gapsize)
    return 0.5

ctx.bcscore = bcscore
ctx.bcscore.coor1 = ctx.coor1
ctx.bcscore.coor2 = ctx.coor2
ctx.bcscore.flanksize = 4
ctx.bcscore.gapsize = 7

ctx.bc_result = ctx.bcscore
await ctx.computation()
print(ctx.bc_result.value)

Execute local
Execute local
Execute local
BCSCORE 4 7
Waiting for: [Seamless transformer: .bcscore.tf]
0.5


In [4]:
ctx.bcscore.language = "c"

ctx.bcscore.inp.schema.mount("bcscore-schema.json", "r", authority="file")
ctx.bcscore.code.mount("bcscore.c", authority="file")
ctx.bcscore.result.example.set(1.0)
ctx.bcscore.result.schema["form"] = {"bytesize": 4}
await ctx.translation()

ctx.bcscore.main_module.bclib.language = "c"
ctx.bclib_code >> ctx.bcscore.main_module.bclib.code 
ctx.bclib_code.mount("BClib.c")

Execute local
Execute local


************************************************************
Exception in Seamless transformer: .bcscore.tf.gen_header, stream element None:
Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/jobscheduler.py", line 236, in _execute_local
    raise Exception(msg)
Exception: Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/execute.py", line 79, in execute
    exec_code(code, identifier, namespace, inputs, output_name)
  File "/home/sjoerd/seamless/seamless/core/cached_compile.py", line 58, in exec_code
    exec(code_obj, namespace)
  File "Seamless transformer: .bcscore.tf.gen_header", line 16, in <module>
    raise TypeError("Input schema needs to be defined")
TypeError: Input schema needs to be defined

************************************************************



Execute local


************************************************************
Exception in Seamless transformer: .bcscore.tf.gen_header, stream element None:
Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/jobscheduler.py", line 236, in _execute_local
    raise Exception(msg)
Exception: Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/execute.py", line 79, in execute
    exec_code(code, identifier, namespace, inputs, output_name)
  File "/home/sjoerd/seamless/seamless/core/cached_compile.py", line 58, in exec_code
    exec(code_obj, namespace)
  File "Seamless transformer: .bcscore.tf.gen_header", line 16, in <module>
    raise TypeError("Input schema needs to be defined")
TypeError: Input schema needs to be defined

************************************************************



In [5]:
await ctx.computation()
ctx.bcscore.status

Execute local
Execute local
gcc -c bclib.c -O3 -ffast-math -march=native -fPIC -p -g -o bclib.o

gcc -c main.c -O3 -ffast-math -march=native -fPIC -p -g -o main.o



************************************************************
Exception in Seamless transformer: .bcscore.tf.translator, stream element None:
Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/jobscheduler.py", line 208, in _execute_local
    mod = build_module(value)
  File "/home/sjoerd/seamless/seamless/core/build_module.py", line 140, in build_module
    full_module_name, completed_checksum, completed_module_definition
  File "/home/sjoerd/seamless/seamless/core/build_module.py", line 124, in build_compiled_module
    mod = import_extension_module(full_module_name, module_code, debug, source_files)
  File "/home/sjoerd/seamless/seamless/core/build_module.py", line 54, in import_extension_module
    importlib.import_module(full_module_name)
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<fro

Execute local
gcc -c bclib.c -O3 -ffast-math -march=native -fPIC -p -g -o bclib.o

gcc -c main.c -O3 -ffast-math -march=native -fPIC -p -g -o main.o

Execute local
gcc -c bclib.c -O3 -ffast-math -march=native -fPIC -p -g -o bclib.o

gcc -c main.c -O3 -ffast-math -march=native -fPIC -p -g -o main.o


************************************************************
Exception in Seamless transformer: .bcscore.tf.translator, stream element None:
Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/jobscheduler.py", line 208, in _execute_local
    mod = build_module(value)
  File "/home/sjoerd/seamless/seamless/core/build_module.py", line 140, in build_module
    full_module_name, completed_checksum, completed_module_definition
  File "/home/sjoerd/seamless/seamless/core/build_module.py", line 124, in build_compiled_module
    mod = import_extension_module(full_module_name, module_code, debug, source_files)
  File "/home/sjoerd/seamless/seamless/core/build_module.py", line 54, in import_extension_module
    importlib.import_module(full_module_name)
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<fro


Execute local
Waiting for: [Seamless transformer: .bcscore.tf.translator]
Execute local
gcc -c bclib.c -O3 -ffast-math -march=native -fPIC -p -g -o bclib.o

gcc -c main.c -O3 -ffast-math -march=native -fPIC -p -g -o main.o



'OK'

In [6]:
import numpy as np

def validate_shape(self):
    assert self.coor1.shape[0] == 2 * self.flanksize, self.coor1.shape
    assert self.coor2.shape[0] == 2 * self.flanksize + self.gapsize, self.coor2.shape

example = ctx.bcscore.example
with example.fork() as f:
    example.coor1 = np.zeros((10, 3))
    example.coor2 = np.zeros((14, 3))
    example.flanksize = 5
    example.gapsize = 4
    example.add_validator(validate_shape, name="validate_shape")

In [7]:
await ctx.computation()
print(ctx.bcscore.schema)
print(ctx.bcscore.result.schema)
print(ctx.bcscore.header.value)

{'properties': {'coor1': {'form': {'contiguous': True,
                                   'ndim': 2,
                                   'shape': [-1, 3]},
                          'items': {'form': {'bytesize': 8, 'type': 'number'}},
                          'storage': 'binary',
                          'type': 'array'},
                'coor2': {'form': {'contiguous': True,
                                   'ndim': 2,
                                   'shape': [-1, 3]},
                          'items': {'form': {'bytesize': 8, 'type': 'number'}},
                          'storage': 'binary',
                          'type': 'array'},
                'flanksize': {'type': 'integer'},
                'gapsize': {'type': 'integer'}},
 'type': 'object',
 'validators': [{'code': 'def validate_shape(self):\n'
                         '    assert self.coor1.shape[0] == 2 * '
                         'self.flanksize, self.coor1.shape\n'
                         '    assert self.coor2

In [8]:
await ctx.computation()
print(ctx.bc_result.value)

0.9981744289398193
